In [2]:
# 编写一个斐波拉切C扩展函数
# python中的实现
def fibonacci(n):
    if n < 2:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

In [1]:
# 纯C扩展 - 详见chapter4文件夹
# 构建指令：pip install -e <file folder>
from chapter4 import fibonacci
help(fibonacci.fibonacci)

Help on built-in function fibonacci in module chapter4.fibonacci:

fibonacci(...)
    fibonacci(n): Return nth Fibonacci sequence numbercomputed recursively



In [4]:
[fibonacci.fibonacci(n) for n in range(10)]

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [2]:
fibonacci.fibonacci(-1)

ValueError: n must not be less than 0

In [2]:
# Cython作为源码编译器 - 使用纯python代码创建扩展
# 见文件夹fibonacci
from fibonacci import fibonacci

In [4]:
fibonacci(8)

34

In [ ]:
# 分发Cython扩展的最佳实践
import os

from distutils.core import setup
from distutils.extension import Extension


try:
    # 只有当Cython可用时
    # cython源到源的编译才可以用
    import Cython
    # 并且特定的环境变量明确说明
    USER_CYTHON = bool(os.environ.get("USE_CYTHON"))

except ImportError:
    USER_CYTHON = False

ext = '.pyx' if USER_CYTHON else '.c'

extensions = [Extension('fibonacci', ['fibonacci' + ext])]

if USER_CYTHON:
    from Cython.Build import cythonize
    extensions = cythonize(extensions)

setup(
    name='fibonacci',
    ext_modules=extensions,
    requires=['cython==0.23.4']
)

In [11]:
# Cython作为一门语言 - 见fibonacci.pyx
from fibonacci import fibonacci
fibonacci(5)

8

In [12]:
fibonacci(-8)

1

In [13]:
fibonacci(10**10)

RecursionError: maximum recursion depth exceeded in comparison

In [ ]:
# 从标准C库中用一个qsort()函数对一个随机乱序的整数数列进行排序
from random import shuffle

import ctypes
from ctypes.util import find_library


libc = ctypes.windll.LoadLibrary(find_library('c'))

CMPFUNC = ctypes.WINFUNCTYPE(
    # 返回类型
    ctypes.c_int,
    # 接受的第一个参数
    ctypes.POINTER(ctypes.c_int),
    # 接受的第二个参数
    ctypes.POINTER(ctypes.c_int)
)

def ctypes_int_compare(a, b):
    # 参数是指针类型，所以可以通过索引访问
    print(f'{a[0]} cmp {b[0]}')
    return a[0] - b[0]

def main():
    numbers = list(range(5))
    shuffle(numbers)
    print(f'shuffled: {numbers}')

    # 创建一个代表数组的新类型，它和numbers列表有相同的长度
    NumbersArray = ctypes.c_int * len(numbers)
    # 使用新类型创建一个新的C数组
    c_array = NumbersArray(*numbers)

    libc.qsort(
        # 指向被排序的数组的指针
        c_array,
        # 数组长度
        len(c_array),
        # 数组中单个元素的大小
        ctypes.sizeof(ctypes.c_int),
        # 回调（指向c比较函数的指针）
        CMPFUNC(ctypes_int_compare)
    )
    print(f'sorted: {list(c_array)}')

main()